#### Script to check rf and compute the visuomemory index block 1
- First, check if neuron responding to the task by comparing activity during trial vs baseline
- Second, check whether the rf is in in or out
- Third, compute the visuomemory index

In [6]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import signal,stats
import pandas as pd
from typing import Dict
from collections import defaultdict
from ephysvibe.trials.spikes import firing_rate,sp_constants,plot_raster
from ephysvibe.trials import select_trials
from ephysvibe.spike_sorting import config
from ephysvibe.structures.bhv_data import BhvData
from ephysvibe.structures.spike_data import SpikeData
from ephysvibe.analysis import raw_ch,circular_stats
from ephysvibe.task import task_constants,def_task
import os 
seed=2023

### Def functions

In [7]:
def get_neurons_info(
    sp_samples: np.ndarray,
    neuron_type: np.ndarray,
    task: pd.DataFrame,
    dur_fix: int,
    st_v: int,
    dur_v: int,
    st_d: int,
    end_d: int,
    st_t: int,
    end_t: int,
    clusterdepth: np.ndarray,
    date: str,
    min_trials: int = 3,
    n_spikes: int = 1,
) -> pd.DataFrame:
    # trial_dur = sp_samples.shape[2]
    neurons_info: Dict[str, list] = defaultdict(list)
    i_good, i_mua, n_type = 0, 0, 0
    for i_neuron, type_neuron in enumerate(neuron_type):
        # check and count type of unit
        if type_neuron == "good":
            i_good += 1
            n_type = i_good
        elif type_neuron == "mua":
            i_mua += 1
            n_type = i_mua
        max_in_out = np.array([0, 0])
        larger, v_larger, d_larger, idx_p_min = (
            np.array([False, False]),
            np.array([False, False]),
            np.array([False, False]),
            np.array([0, 0]),
        )
        t_larger = np.nan
        p_in_out, p_v, p_m, vm_index = (
            np.array([None, None]),
            np.array([None, None]),
            np.array([None, None]),
            np.nan,
        )
        tr_min, tr_max, mean_delay_all, mean_visual_all = (
            np.array([np.nan, np.nan]),
            np.array([np.nan, np.nan]),
            np.array([np.nan, np.nan]),
            np.array([np.nan, np.nan]),
        )
        for i, in_out in enumerate(["in", "out"]):  # iterate by code'
            trial_idx = task[
                (task["i_neuron"] == i_neuron)
                & (task["in_out"] == in_out)
                & (task["sample"] != "o0_c0")
            ]["trial_idx"]
            trial_idx = trial_idx[(sp_samples[trial_idx, i_neuron].sum(axis=1) >= n_spikes)]
            n_tr = len(trial_idx)
            true_in_out = "in"
            ## Neurons respnding to the task
            if n_tr > min_trials:
                mean_visual = sp_samples[
                    trial_idx, i_neuron, dur_fix + st_v : dur_fix + dur_v
                ].mean(axis=1)
                mean_delay = sp_samples[
                    trial_idx, i_neuron, dur_fix + st_d : dur_fix + end_d
                ].mean(axis=1)
                mean_test = sp_samples[
                    trial_idx, i_neuron, dur_fix + st_t : dur_fix + end_t
                ].mean(axis=1)
                mean_bl = sp_samples[trial_idx, i_neuron, :dur_fix].mean(axis=1)
                v_larger[i] = mean_bl.mean() < mean_visual.mean()
                d_larger[i] = mean_bl.mean() < mean_delay.mean()
                t_larger = mean_bl.mean() < mean_test.mean()
                larger[i] = v_larger[i] or d_larger[i] or t_larger
                # paired sample t-test
                p_v[i] = stats.ttest_rel(mean_bl, mean_visual)[1]
                p_m[i] = stats.ttest_rel(mean_bl, mean_delay)[1]
                p_t = stats.ttest_rel(mean_bl, mean_test)[1]

                p_in_out[i] = np.min([p_v[i], p_m[i], p_t])
                max_in_out[i] = np.max([v_larger[i], d_larger[i], t_larger])
                idx_p_min[i] = np.argmin([p_v[i], p_m[i], p_t])

                all_mean = sp_samples[trial_idx, i_neuron, : dur_fix + end_d].mean(axis=0)
                tr_min[i], tr_max[i] = np.min(all_mean), np.max(all_mean)
                mean_delay_all[i], mean_visual_all[i] = (
                    mean_delay.mean(),
                    mean_visual.mean(),
                )
        # Get receptive field
        if not np.all(np.isnan(tr_min)):
            tr_max_all = np.nanmax(tr_max)  # - tr_min_all
            if not(p_v[0] is None):
                p_and_large_in = np.any(
                    np.logical_and(
                        np.array([p_v[0], p_m[0]]) < 0.05,
                        np.array([v_larger[0], d_larger[0]]),
                    )
                )
            if not(p_v[0] is None) and p_and_large_in:
                mean_delay = (mean_delay_all[0]) / tr_max_all  # - tr_min_all
                mean_visual = (mean_visual_all[0]) / tr_max_all  # - tr_min_all
                vm_index = (mean_delay - mean_visual) / (mean_delay + mean_visual)
            else:
                if not(p_v[1] is None):
                    p_and_large_out = np.any(
                        np.logical_and(
                            np.array([p_v[1], p_m[1]]) < 0.05,
                            np.array([v_larger[1], d_larger[1]]),
                        )
                    )
                if not(p_v[1] is None) and p_and_large_out:
                    true_in_out = "out"
                    mean_delay = (mean_delay_all[1]) / tr_max_all  # - tr_min_all
                    mean_visual = (mean_visual_all[1]) / tr_max_all  # - tr_min_all
                    vm_index = (mean_delay - mean_visual) / (mean_delay + mean_visual)

        for i, in_out in enumerate(["in", "out"]):  # iterate by code'
            neurons_info["array_position"] += [i_neuron]
            neurons_info["cluster"] += [n_type]
            neurons_info["group"] += [type_neuron]
            neurons_info["in_out"] += [in_out]
            neurons_info["true_in_out"] += [true_in_out]
            neurons_info["vm_index"] += [vm_index]
            neurons_info["p"] += [p_in_out[1]]
            neurons_info["larger"] += [larger[i]]
            neurons_info["v_larger"] += [v_larger[i]]
            neurons_info["p_v"] += [p_v[i]]
            neurons_info["d_larger"] += [d_larger[i]]
            neurons_info["p_m"] += [p_m[i]]
            neurons_info["depth"] += [clusterdepth[i_neuron]]
            neurons_info["date"] += [date]
    neurons_info = pd.DataFrame(neurons_info)
    return neurons_info

### Start analysis

In [8]:
# load data
filepath = "/envau/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/session_struct/v4/spikes/2023-02-09_10-42-54_Riesling_v4_e1_r1_sp.h5"  #2023-02-27_10-50-03 2023-02-09_10-42-54 2023-03-21_10-40-02
bhv_path = '/envau/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/session_struct/bhv/2023-02-09_10-42-54_Riesling_e1_r1_bhv.h5'
s_path = os.path.normpath(filepath).split(os.sep)
data = SpikeData.from_python_hdf5(filepath)
bhv = BhvData.from_python_hdf5(bhv_path)

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/envau/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/session_struct/v4/spikes/2023-02-09_10-42-54_Riesling_v4_e1_r1_sp.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
# Select trials and create task frame
block = 1
trials_block = np.where(np.logical_and(data.trial_error == 0, data.block == block))[0]
print("Number of clusters: %d" % len(data.clustersgroup))

Number of clusters: 47


In [ ]:
if np.any(np.isnan(data.neuron_cond)):
    neuron_cond = np.ones(len(data.clustersgroup))
else:
    neuron_cond = data.neuron_cond
task = def_task.create_task_frame(
    condition=bhv.condition[trials_block],
    test_stimuli=bhv.test_stimuli[trials_block],
    samples_cond=task_constants.SAMPLES_COND,
    neuron_cond=neuron_cond,
)

In [ ]:
sp_samples = data.sp_samples[trials_block]
neuron_type = data.clustersgroup
code_samples = data.code_samples[trials_block]
code_numbers = data.code_numbers[trials_block]
sp_samples = data.sp_samples[trials_block]

#### Select parameters

In [ ]:
# Timings 
## fixation
dur_fix = 200
## visual stim
st_v =   50
dur_v = 250
## delay
st_d = 500 
end_d = 500 +400
## test
st_t = 950
end_t = 950 + 400
# trials and threshold
min_trials = 3
n_spikes = 1
p_threshold = 0.05
vm_threshold = 0.4

In [ ]:
shifts_on = code_samples[:,4]  
shifts_on = shifts_on[:,np.newaxis]
shifts_on = np.where(np.isnan(shifts_on),0,shifts_on)
shifts_test = code_samples[:,6]
shifts_test = shifts_test[:,np.newaxis]
shifts_test = np.where(np.isnan(shifts_test),0,shifts_test)

sp_shift_on = SpikeData.indep_roll(sp_samples, -(shifts_on-dur_fix).astype(int), axis=2)[:,:,:end_d+dur_fix]
sp_shift_test = SpikeData.indep_roll(sp_samples, -(shifts_test).astype(int), axis=2)[:,:,:end_t-st_t]
sp_shift = np.concatenate([sp_shift_on,sp_shift_test],axis=2)

#### Select responding neurons and check rf

In [ ]:
task

,i_neuron,trial_idx,sample,in_out,n_test_stimuli,code
0,0,3,o1_c1,in,2,2
1,0,9,o1_c1,in,2,2
2,0,11,o1_c1,in,2,2
3,0,18,o1_c1,in,2,2
4,0,46,o1_c1,in,2,2
...,...,...,...,...,...,...
15834,46,321,o5_c5,out,1,1
15835,46,325,o5_c5,out,4,4
15836,46,328,o5_c5,out,3,3
15837,46,333,o5_c5,out,5,6


In [ ]:
neurons_info = get_neurons_info(
    sp_shift,
    neuron_type,
    task,
    dur_fix=dur_fix,
    st_v = st_v,
    dur_v=dur_v,
    st_d=st_d,
    end_d=end_d,
    st_t=st_t,
    end_t=end_t,
    min_trials=min_trials,
    n_spikes = n_spikes,
    clusterdepth=data.clusterdepth,
    date=s_path[-1][:19],
)
neurons_info = neurons_info[(neurons_info['p']<0.05) & (neurons_info['larger']==True)] # responding neurons


#### Compute vm index

In [ ]:
neurons_info = neurons_info[neurons_info["in_out"]==neurons_info["true_in_out"]]
neurons_info = neurons_info[(np.logical_and(neurons_info["p_v"]<0.05,neurons_info["v_larger"]==True)) | (np.logical_and(neurons_info["p_m"]<0.05,neurons_info["d_larger"]==True))]

In [ ]:
neurons_info

,array_position,cluster,group,in_out,true_in_out,vm_index,p,larger,v_larger,p_v,m_larger,p_m,depth,date
8,4,4,mua,in,in,-0.035496,2.974447e-09,True,True,6.221785e-06,True,1.202486e-04,38000.0,2023-02-09_10-42-54
16,8,7,mua,in,in,-0.391937,8.928448e-10,True,True,1.839190e-19,False,7.295938e-06,46000.0,2023-02-09_10-42-54
20,10,9,mua,in,in,-0.460765,4.135747e-03,True,True,3.496508e-07,False,3.083665e-03,49000.0,2023-02-09_10-42-54
22,11,10,mua,in,in,-0.532419,8.469441e-04,True,True,3.405840e-26,True,6.833794e-01,52000.0,2023-02-09_10-42-54
26,13,12,mua,in,in,-0.535981,4.721518e-06,True,True,2.301518e-10,False,2.186293e-07,53000.0,2023-02-09_10-42-54
33,16,3,good,out,out,0.114317,4.869865e-08,True,True,3.945785e-02,True,4.869865e-08,57000.0,2023-02-09_10-42-54
34,17,15,mua,in,in,0.150000,9.994835e-03,True,True,4.071592e-01,True,2.942880e-03,58000.0,2023-02-09_10-42-54
38,19,17,mua,in,in,0.163504,1.675441e-04,True,False,1.932225e-05,True,3.243163e-04,60000.0,2023-02-09_10-42-54
40,20,18,mua,in,in,-0.035000,9.944321e-04,True,True,4.619027e-02,True,2.317300e-01,62000.0,2023-02-09_10-42-54
44,22,20,mua,in,in,0.797753,8.848563e-25,True,False,5.192670e-05,True,1.313550e-39,66000.0,2023-02-09_10-42-54
